In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import welch, lfilter
from scipy import fftpack
from scipy.fft import fft

In [ ]:
def plot_time_domain_channels_stereo(time, lc, rc):
    #Plota as figuras ao longo do tempo
    #Plota os canais esquerdo e direito
    plt.figure(1,figsize=(20, 5))
    plt.plot(time, lc, label="Canal esquerdo")
    plt.legend()
    plt.xlabel("Tempo [s]")
    plt.ylabel("Amplitude")
    plt.show()

    plt.figure(2,figsize=(20, 5))
    plt.plot(time, rc, color="red",label="Canal direito")
    plt.legend()
    plt.xlabel("Tempo [s]")
    plt.ylabel("Amplitude")
    plt.show()

In [ ]:
def plot_spect_welch_channels_stereo(lc, rc, fs):
    #Sample Frequencies, Power Spectral Density
    sf_lc, psd_lc = welch(
        x=lc, 
        fs=fs, 
        window='flattop', 
        nperseg=512, 
        scaling='spectrum'
    )
    sf_rc, psd_rc = welch(
        x=rc, 
        fs=fs, 
        window='flattop', 
        nperseg=512, 
        scaling='spectrum'
    )

    #Plota o espectro do sinal para frequencias normalizadas entre 0 1 pi 
    #(frequencias positivas)
    plt.subplots(figsize=(15,5))
    plt.subplot(1, 2, 1)
    plt.semilogy(sf_lc, psd_lc, label="Canal esquerdo")
    plt.legend()
    plt.xlabel('Frequencia [rad]')
    plt.ylabel('Espectro')

    plt.subplot(1, 2, 2)
    plt.semilogy(sf_rc, psd_rc, color="red", label="Canal direito")
    plt.legend()
    plt.xlabel('Frequencia [rad]')
    plt.ylabel('Espectro')
    plt.show()

In [ ]:
def plot_spect_fft_channels_stereo(lc,rc, n, t):
    freq_lc = np.linspace(0.0, 1.0//(2.0*t), n//2) #Interpola para determinar eixo da frequencia
    sig_fft_lc = fftpack.fft(lc)
    plt.figure(3, figsize=(20,5))    
    plt.title("Canal esquerdo")
    plt.plot(freq_lc, 2.0/n * np.abs(sig_fft_lc[0:n//2]), label="Canal esquerdo")
    plt.legend()
    plt.xlabel('Frequencia [Hz]')
    plt.ylabel('Espectro de amplitudes')
    plt.legend()

    freq_rc = np.linspace(0.0, 1.0//(2.0*t), n//2) #Interpola para determinar eixo da frequencia
    sig_fft_rc = fftpack.fft(rc)
    plt.figure(4, figsize=(20,5))
    plt.title("Canal direito")
    plt.plot(freq_rc, 2.0/n * np.abs(sig_fft_rc[0:n//2]), color="red", label="Canal direito")
    plt.legend()
    plt.xlabel('Frequencia [Hz]')
    plt.ylabel('Espectro de amplitudes')
    plt.legend()
    plt.show()
    return freq_lc, sig_fft_lc, freq_rc, sig_fft_lc

In [ ]:
#Carrega o arquivo
sampling_rate, data = wavfile.read('569127__josefpres__dark-loops-201-simple-mix-2-short-loop-60-bpm.wav')
#sampling_rate, data = wavfile.read('581010__xcreenplay__smoking-in-the-angel-section2.wav')

number_of_samples = data.shape[0]
number_of_channels = data.shape[1]

#Tempo total = numero de amostras / fs
duration = number_of_samples / sampling_rate

#Carrega o arquivo em dois canais (audio estereo)
left_channel = data[:, 0]
right_channel  = data[:, 1]

print(f"Numero de canais = {number_of_channels}")
print(f"Duracao = {duration}s")
print(f'Numero de amostras: {number_of_samples}')
print(f"Amostras por segundo: {sampling_rate}Hz")


# Plote o sinal no domínio do tempo.

In [ ]:
#Interpola para determinar eixo do tempo
time = np.linspace(0., duration, number_of_samples)
plot_time_domain_channels_stereo(
    time=time,
    lc=left_channel,
    rc=right_channel
)

# Plote o espectro de frequências do sinal e no domínio do tempo para as primeiras N amostras. 

In [ ]:
N = 100000 #Numero de Amostras escolhidos
T = time[1] - time[0] # sample spacing
plot_time_domain_channels_stereo(
    time=time[0:N],
    lc=left_channel[0:N],
    rc=right_channel[0:N]
)
plot_spect_welch_channels_stereo(
    lc  = left_channel[0:N],
    rc  = right_channel[0:N],
    fs = (2*np.pi)
)
return_amost_n = plot_spect_fft_channels_stereo(
    lc  = left_channel[0:N],
    rc  = right_channel[0:N],
    n=N,
    t=T
)

# Projeto de um filtro digital passa-baixas com resposta ao impulsofinita (FIR) que corta a metade do conteúdo espectral do arquivo de áudio

In [ ]:
freq_lc = np.linspace(0.0, 1.0//(2.0*T), number_of_samples//2) #Interpola para determinar eixo da frequencia
sig_fft_lc = fftpack.fft(left_channel)

plt.figure(1,figsize=(15, 5))
plt.plot(freq_lc, 2.0/number_of_samples * np.abs(sig_fft_lc[:number_of_samples//2]))
#"centro de massa" da função no domínio da frequência 
peso_lc = 0.0 #porcentagem do sinal acumulado
rangeData_lc = len(sig_fft_lc)//2 #frequências positivas da FFT 
sc_lc = sum(abs(sig_fft_lc[:rangeData_lc])) #todo o sinal acumulado 
count_lc = 0 # quantidade de amostras somadas

for i in abs(sig_fft_lc[:rangeData_lc]):
  peso_lc = peso_lc + i/sc_lc
  count_lc = count_lc + 1
  if(peso_lc >= 0.5):
    print(peso_lc)
    break
print(freq_lc[count_lc]) #frequência de corte para o primeiro canal

plt.title("Canal Esquerdo")
plt.axvline(x = freq_lc[count_lc], color = 'black', ls = '--')
plt.show()


freq_rc = np.linspace(0.0, 1.0//(2.0*T), number_of_samples//2) #Interpola para determinar eixo da frequencia
sig_fft_rc = fftpack.fft(right_channel)

plt.figure(2,figsize=(15, 5))
plt.plot(freq_rc, 2.0/number_of_samples * np.abs(sig_fft_rc[:number_of_samples//2]), color='red')

peso_rc = 0.0 
rangeData_rc = len(sig_fft_rc)//2 
sc_rc = sum(abs(sig_fft_rc[:rangeData_rc])) 
count_rc = 0 

for i in abs(sig_fft_rc[:rangeData_rc]):
  peso_rc = peso_rc + i/sc_rc
  count_rc = count_rc + 1
  if(peso_rc >= 0.5):
    print(peso_rc)
    break
print(freq_rc[count_rc])

plt.title("Canal Direito")
plt.axvline(x = freq_rc[count_rc], color = 'black', ls = '--')
plt.show()

In [ ]:
aux = (2.0/number_of_samples * np.abs(sig_fft_lc[:number_of_samples//2])).cumsum()
half = aux[-1] / 2
value_half = aux[aux >= half][0]
freq_half = freq_lc[aux >= value_half][0]
print("Frequência de metade do conteúdo espectral: ", freq_half, " Hz")

In [ ]:
aux = (2.0/number_of_samples * np.abs(sig_fft_rc[:number_of_samples//2])).cumsum()
half = aux[-1] / 2
value_half = aux[aux >= half][0]
freq_half = freq_rc[aux >= value_half][0]
print("Frequência de metade do conteúdo espectral: ", freq_half, " Hz")

## Coeficientes do filtro projeto no PyFDA

In [ ]:
#Carrega os coeficientes do filtro
b = np.genfromtxt('coeffs_pyfda_lp.csv', delimiter=',')
#Plota coeficientes do filtro FIR
plt.figure(7, figsize=(20, 5))
plt.title("Filtro FIR projetado")
plt.stem(b)
plt.show()

## Implemente a filtragem com a operação de convolução no domínio da frequência ( Método de sobreposição e soma)

In [ ]:
def sum_cov(in_signal, filter):
    signal_size = len(in_signal)
    out_signal = [0 for i in range(signal_size)]
    for n in range(signal_size):
        sum = 0
        for k in range(len(filter)):
            sum += in_signal[n-k] * filter[k]
        out_signal[n] = sum
    return out_signal

In [ ]:
#Filtra os dados dos canais esquerdo e direito
filtered_lc = sum_cov(left_channel[0:N], b)
filtered_rc = sum_cov(right_channel[0:N], b)

In [ ]:
# Escrita de arquivo filtrado
audio = np.array([filtered_lc, filtered_rc]).T
scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
filename = 'signal_filtered' + '.wav'
wavfile.write(filename, sampling_rate, scaled)

In [ ]:
#Dizimando o sinal pelo fator M
M=2

decimated_lc = filtered_lc[0:-1:M]
decimated_rc = filtered_rc[0:-1:M]

# Plot os conteúdos espectrais (filtrados e dizimados)

In [ ]:
print(return_amost_n)

In [ ]:
print("Filtrados")
plot_spect_welch_channels_stereo(
    lc  = filtered_lc,
    rc  = filtered_rc,
    fs = (2*np.pi)
)

return_filtered = plot_spect_fft_channels_stereo(
    lc  = filtered_lc,
    rc  = filtered_rc,
    n=N,
    t=T
)

print("Dizimados")
plot_spect_welch_channels_stereo(
    lc  = decimated_lc,
    rc  = decimated_rc,
    fs = (2*np.pi)/M
)

return_decimated = plot_spect_fft_channels_stereo(
    lc  = decimated_lc,
    rc  = decimated_rc,
    n=N//M,
    t=T
)



In [ ]:
print("Parte filtrada")
pf = abs(return_amost_n[1])- abs(return_filtered[1])
xf2 = np.linspace(0.0, 1.0//(2*T), N//2)
plt.figure(1)
plt.plot(xf2, 2.0/N * np.abs(pf[:N//2]))

In [ ]:
# Escrita de arquivo dizimado
audio = np.array([decimated_lc, decimated_rc]).T
scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
filename = 'signal_decimated_' + str(M) + '.wav'
wavfile.write(filename, sampling_rate//M, scaled)

In [ ]:
filtered_time_domain_lc = fftpack.ifft(filtered_lc)
filtered_time_domain_rc = fftpack.ifft(filtered_rc)

decimated_time_domain_lc = fftpack.ifft(decimated_lc)
decimated_time_domain_rc = fftpack.ifft(decimated_rc)

# Plot os conteúdos temporais (filtrados e dizimados)

In [ ]:
plot_time_domain_channels_stereo(time[0:N],filtered_time_domain_lc,filtered_time_domain_rc)
plot_time_domain_channels_stereo(time[0:N-1:M], decimated_time_domain_lc, decimated_time_domain_rc)